In [60]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [35]:
from pathlib import Path
import os
os.getcwd().split()


'c:\\Users\\HP\\MedicalChatbot\\research'

In [2]:
PINECONE_API_KEY=""
PINECONE_API_ENV=""

In [8]:
def pdf_load(data):
    loader=DirectoryLoader(data,glob="*.pdf",
                    loader_cls=PyPDFLoader)
    document= loader.load()
    return document

In [37]:
#extractdata= pdf_load("c:\\Users\\HP\\MedicalChatbot\\pdf_data\\")
extractdata= pdf_load("pdf_data\\")

In [7]:
extractdata

[]

In [52]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [53]:
text_chunk =text_split(extractdata)

In [15]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [41]:
embedding = download_hugging_face_embeddings()

In [42]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [43]:
query_result = embedding.embed_query("Hello World")

In [45]:
len(query_result)

384

In [56]:
#Initializing the Pinecone

pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="test"
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunk], embedding, index_name=index_name)


In [57]:
#If we already have an index we can load it like this
#docsearch=Pinecone.from_existing_index(index_name, embedding)

query = "What are Diagnosis"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [62]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [68]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="model\\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})



In [69]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  You're right, migraines usually cause a throbbing pain on one side of the head, and there is no evidence to suggest that head injuries directly cause brain tumors. However, researchers are investigating the potential relationship between brain tumors and viruses, family history, and long-term exposure to electromagnetic fields. If you're experiencing persistent or severe headaches, it's important to consult with a medical professional to rule out any underlying conditions.
Response :  Diagnosis is a medical term that refers to the process of identifying or confirming the presence of a disease or condition based on its symptoms, signs, test results, and other clinical findings. The goal of diagnosis is to provide an accurate and timely identification of the underlying cause of a patient's symptoms or conditions, which can help guide appropriate treatment and improve outcomes. In some cases, a definitive diagnosis may not be possible, but a provisional diagnosis can be made b

KeyboardInterrupt: 